In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv('/kaggle/input/avocado-prices/avocado.csv')
data.head(10)

In [ ]:
data.info()

Checking for missing values

In [ ]:
data.isnull().sum()

Converting date to datetime object

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])

There are 2 types of avocados : conventional and organic. For this example, let us consider organic.

In [ ]:
data.type.value_counts()

There are 54 different regions.

In [ ]:
data.region.nunique()
data.region.value_counts().sort_index()

****We will forecast the number of organic avocados that will sell in Albany.**

In [ ]:
sales = data[(data['region']=='Albany')&(data['type']=='organic')][['Date','Total Bags']]

In [ ]:
sales = sales.sort_values(by='Date')

Formatting columns for inputting into Facebook prophet

In [ ]:
sales = sales.rename(columns = {'Date':'ds','Total Bags':'y'})
sales = sales.reset_index(drop=True)
sales.head(6)

In [ ]:
#plot daily sales
pd.plotting.register_matplotlib_converters()
ax = sales.set_index('ds').plot(figsize=(12,4))
ax.set_ylabel('Daily number of sales')
ax.set_xlabel('Date')
plt.show()

Let us account for holidays in the model. Go ahead and download the holidays data set from below.
https://data.world/sudipta/us-federal-holidays-2011-2020

In [ ]:
import pandas as pd
holidays = pd.read_csv('/kaggle/input/holidays/us-federal-holidays-2011-2020.csv')

In [ ]:
holidays.head()

In [ ]:
holidays.Date = pd.to_datetime(holidays.Date)

In [ ]:
holidays = holidays.rename(columns={'Date':'ds','Holiday':'holiday'})
holidays = holidays[['holiday','ds']]
holidays.head()

In [ ]:
from fbprophet import Prophet

#set uncertanity interval to 95%
mymodel = Prophet(interval_width  = 0.95, holidays = holidays)
mymodel.fit(sales)

# dataframe that extends into future 6 weeks 
future_dates = mymodel.make_future_dataframe(periods = 6*7)

print("First week to forecast.")
future_dates.tail(7)

In [ ]:
#Predictions
forecast = mymodel.predict(future_dates)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
mymodel.plot(forecast);

In [ ]:
mymodel.plot_components(forecast);

The trend seems to increase as the years go ahead. It looks like we dont have any holidays in the Albany dataset. In the third graph we can see the yearly trends, where the sales peak towards the end of October and go south towards the end of November.

**For visualizing these plots for different regions and type of avocado use the code below:**

In [ ]:
data['region'].unique()

In [ ]:
print('Enter region: ')
region = input()
print('\n')
print('choose conventional/organic: ')
type_ = input()

sales = data[(data['region']==region)&(data['type']==type_)][['Date','Total Bags']]
sales = sales.sort_values(by='Date')

sales = sales.rename(columns = {'Date':'ds','Total Bags':'y'})
sales = sales.reset_index(drop=True)

pd.plotting.register_matplotlib_converters()
ax = sales.set_index('ds').plot(figsize=(12,4))
ax.set_ylabel('Daily number of sales')
ax.set_xlabel('Date')
plt.show()
mymodel = Prophet(interval_width  = 0.95, holidays = holidays)
mymodel.fit(sales)

future_dates = mymodel.make_future_dataframe(periods = 6*7)
forecast = mymodel.predict(future_dates)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

print('FORECAST MODELS')
mymodel.plot(forecast);
mymodel.plot_components(forecast);